<a href="https://colab.research.google.com/github/anthonyhu25/Variance-Reduction-Independent-Metropolis/blob/main/Variance_Reduction_Independent_Metropolis_Example_3_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import random
from numpy import linalg
import math
import scipy
import scipy.stats
import matplotlib.pyplot as plt
from scipy.stats import rv_continuous, rv_discrete
from scipy.stats._distn_infrastructure import rv_frozen
from scipy.special import logsumexp
import scipy.integrate
import warnings
import sys
import statistics
import pandas as pd
from IPython.display import display, Math, HTML

# 3 Adaptive Independent Metropolis Sampler and Numerical Examples

## Overview of Scheme


Suppose we have a sequence of independent Metropolis adaptations $\left\{ IM(P_{i}, \pi, q_{\theta_{i}}) \right\}_{i=1}^{\infty}$ with proposal densities $\left\{q_{\theta_{i}} \right\}_{i=1}^{\infty}$, with parameters $\theta_{i}$ drawn from a vector $𝚯$.

With this new scheme, we add an update function $H_{\theta_{i}}(x)$ and stepsize $\alpha_{i}$ for each step $i$. But for this schema to work, we do need to assume that every sampling chain has a state space $𝕏$ that allows the parameters of the proposal density to converge to some $\theta^{*}$ ($\displaystyle \lim_{i \to \infty} \theta_{i} = \theta^{*}$ in probability for some $\theta^{*} ∈ 𝚯$).

Ideally speaking, the ideal estimator of the $IM(P, \pi, q)$ sampler is the ability to minimize the distance, in terms of the KL divergence, between the proposal density $q$ and the target distribution $\pi$.  

## 3.1 Adaptation based on KL divergence (work in progress)

To obtain a good proposal density $q$ that is as close to possible, in terms of KL divergence, to the target $\pi$, we will use a gradient direction function as our update function $H_{\theta}$ to minmize the KL divergence $𝕂𝕃(q_{\theta}(x)||\pi(x))$. To obtain the sequence of adapted proposal distributions $q_{\theta}(x)$, we update $\theta_{i}$ using this form:

$\theta_{i+1} = \theta_i - \alpha_i \nabla_{\theta_{i}} \mathrm{KL}(q_{\theta_{i}}(x) \,\|\, \pi(x))$ where $\nabla_{\theta_{i}}𝔼(q_{\theta_{i}}(x)||\pi(x))$ = $\nabla_{\theta_{i}}𝔼_{q_{\theta_{i}}}(log\frac{q_{\theta_{i}}(x)}{\pi(x)})$

with $\alpha_{i} > 0$ is a step-size parameter.

However, this optimization procedure is not feasible since the gradient is not available in closed form. Instead, we assume that the independent Metropolis proposal density $q_{\theta_{i}}$ can be reparameterized from a simpler distribution $p(z)$, which allows us to use efficient reparameterization gradient methods, while ensuring that the estimates of the gradient method remain unbiased.

Note that to remain reproducibility (in that I don't have to manually code the gradient of the distributions in the function for each example), I will use PyTorch for this function, as the distributions called from PyTorch have supported gradient function calls. However, PyTorch distribution objects do not have in-built expectation of function calls, something that SciPy objects have. I will call 2 different objects for $q$: a PyTorch distribution object for gradient calculation, and a Scipy distribution object for calculation of $𝔼(q_{\theta_{i}}(x)||\pi(x))$.


### Example 3.3.1: Adaptive IM with d-dimensional Gaussian target and Gaussian proposal

Let our d-dimensional target be the Gaussian $\pi(x) = N(x|0, I_{d})$ and adaptive proposal initialized as $q(x) = N(x| I_{d}, LL^{T})$ where $I_{d}$ is $d$-dimensional length of 1, and L has elements:

$$
L(i,j) =
\begin{cases}
1, & \text{if } i \ge j \\
0, & \text{if } \ i < j\\
\end{cases}
$$

For the unbiased gradient descent method, we can use the reparameterization trick to rewrite the proposal distribution $q$ in terms of a simpler transformation. In this case, let $z \sim N(0_{d}, I_{d})$, where $0_{d}$ is

$H_{\theta}(Y) = \nabla_{\theta}𝔼_{q_{\theta}(x)} log(q_{\theta}(x)) - \nabla_{\theta}log \pi(Y = \mu + Lz), z \sim N(0, I_{d})$, where the update is defined as:

$\theta_{i+1} ← \theta_{i} -\alpha_{i}H_{θ_{i}}(Y_{i})$

Note that $𝔼_{q_{\theta}(x)}log(q_{\theta}(x))$ is the negative entropy of the Gaussian distribution.

The entropy derivation for both univariate and multivariate can be found [here](https://gregorygundersen.com/blog/2020/09/01/gaussian-entropy/).

We will be updating the parameters $(\mu, L)$ in this update function.

For this example, we are using the generated samples from $q$ to estimate the coordinates of the mean vector of the target density $\pi(x)$ (so estimating $\mu_{\pi}(x)$) using the generated samples. So the $F(x)$ function should be the identity function with respect to $x$.

We will rewrite the negative entropy in terms of the Cholesky factor $L$.

$E_{q_{\theta}(x)}log(q_{\theta}(x)) = -H(x) = \frac{D}{2}(1 + log(2\pi)) + \frac{1}{2}log (det(L L^{T}))$

For the gradient of the negative entropy, below are the elementwise derivatives. Note that these schemes will be used for the update for the Adaptive Metropolis

$-\frac{\partial H(x)}{\partial \mu} = 0$

$-2 \frac{\partial H(x)}{ \partial L} = -\frac{\partial}{\partial Σ}(log(det(Σ))) \frac{\partial Σ}{\partial L} = -(\Sigma^{-1})^{T} \frac{\partial Σ}{\partial L} = -\Sigma^{T} \frac{\partial}{\partial L}(LL^{T}) = -(LL^{T})^{-1}[\frac{\partial}{\partial L}(L^{T}) + \frac{\partial}{\partial L}^{T}(L)] \\\ = -(LL^{T})^{-1}2L = -2(L^{T})^{-1}$

And so $\frac{\partial H(x)}{\partial L} = (L^{T})^{-1}$

Note that since $\Sigma$ is symmetric, $\Sigma$ is positive semi-definite, and thus $det(\Sigma)$ = $|det(\Sigma)|$

According to the [Matrix Differential Calculus book](https://nzdr.ru/data/media/biblio/kolxoz/M/MA/MAl/Magnus%20J.,%20Neudecker%20H.%20Matrix%20differential%20calculus%20with%20applications%20in%20statistics%20and%20econometrics%20(3ed.,%20Wiley,%201999)(ISBN%200471986321)(O)(470s)_MAl_.pdf#page=191), we can represent the derivative of $log(det(\Sigma))$ in terms of a trace of its differential and inverse, if we verify that matrix $F = Σ$ is $k$-times continuously differentiable on its domain $S \subset ℝ^{m \times m}$, where $m$ is the dimension of $\Sigma$, and also $log|F| = |log F|$.

Note that in the book, $F$ is treated as a function. Technically speaking, all matrices can be viewed as a linear operator between two spaces. However, in this example, we are simply treating the matrix as itself, and nothing else. Still, this theorem holds.

From the trace trick,

$-2\frac{\partial H(x)}{\partial L} = -2 \frac{\partial H(x)}{\partial \Sigma} \frac{\partial \Sigma}{\partial L} = -tr(Σ^{-1}\frac{\partial \Sigma}{\partial L}) = -tr(Σ^{-1}[\frac{\partial}{\partial L}(LL^{T})])$




In [ ]:
def cholesky_negative_entropy_calculation(L):
  return np.linalg.inv(L.T)

In [ ]:
def multivariate_general_batch_adaptive_gaussian_IMCV_sampler(initial_mu, initial_L, B_batches, l_num_batches, alphas, coefficient_calculation):
  # Sanity check mu and L (Cholesky of proposal covariance matrix) to see if dimensions match
  if len(initial_mu) != initial_L.shape[0] or len(initial_mu) != initial_L.shape[1]:
    print("Dimensions of initial_mu and L(Cholesky factor for covariance matrix) does not match")
    sys.exit(1)
  # Check to see if length of alphas and number of iterations match
  if len(alphas) != B_batches:
    print("Length of alphas and number of iterations do not match")
    sys.exit(1)
  # Initialize list of mus for sampling purposes -- not to be confused with the mu of the gaussian proposal desntiy
  mu_IMCV = []
  mu_IMCV_coefficients = []
  # Need dimension of mu and sigma -- we call this d
  d = len(initial_mu)
  # Initiate the initial parameters (mu, sigma/L) of the proposal -- will need to update throughout the batches
  mu_loop = initial_mu
  L_loop = initial_L
  for i in range(l_num_batches):
    # Initialize the Chain of the sampler
    # X_chain is the current state of chain
    # Y-chain is proposed state of chian
    X_chain = []
    Y_chain = []
    alpha_chain = []
    acceptance_counter = 0
    for j in range(B_batches):
      if len(X_chain) == 0:
        # Sample and accept with probability 1
        ## Using the form in the paper (y = mu + Lz, where z is multivariate standard normal)
        Y_i = mu_loop + L_loop @ scipy.stats.multivariate_normal.rvs(mean = [0 for _ in range(d)], cov = np.eye(d))
        X_chain.append(np.asarray(Y_i))
        Y_chain.append(np.asarray(Y_i))
        alpha_chain.append(1)
      else:
        # Propose sample
        ## Using the form in the paper (y = mu + Lz, where z is multivariate standard normal)
        Y_i = mu_loop + L_loop @ scipy.stats.multivariate_normal.rvs(mean = [0 for _ in range(d)], cov = np.eye(d))
        Y_chain.append(Y_i)
        # Acceptance rejection
        X_i = X_chain[-1]
        # acceptance ratio
        alpha_i_numerator = (scipy.stats.multivariate_normal.pdf(x = Y_i, mean = np.asarray([0 for _ in range(d)]), cov = np.eye(d)) *
              scipy.stats.multivariate_normal.pdf(x = X_i, mean = mu_proposal, cov = sigma_proposal))
        alpha_i_denominator = (scipy.stats.multivariate_normal.pdf(x = X_i, mean = np.asarray([0 for _ in range(d)]), cov = np.eye(d)) *
              scipy.stats.multivariate_normal.pdf(x = Y_i, mean = mu_proposal, cov = sigma_proposal))
        alpha_i = min(1, alpha_i_numerator/alpha_i_denominator)
        alpha_chain.append(alpha_i)
        U = np.random.uniform()
        if U < alpha_i:
          X_chain.append(Y_i)
          acceptance_counter += 1
        else:
          X_chain.append(X_i)
    # End of batch mu calculations and parameter update
    mu_IMCV_i =
    if coefficient_calculation == True:
      print("Work in Progress") # Come back to work on this
    # Update function
    mu_loop =
    L_loop =




